In [18]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import requests
import csv
import os

In [19]:
#import covid data
file_path = "covid_data.csv"
covid_data_df = pd.read_csv(file_path)
covid_data_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [20]:
#check for duplicates

dup_check = covid_data_df[covid_data_df.duplicated(['date', 'state'], keep = False)]


dup_check

,date,state,fips,cases,deaths


No duplicate data

In [21]:
#split date field into year, month, day to calculate monthly totals 

#list for reformatted data
year = []
month = []
day = []
#state_list = []
#monthly_cases = []
#monthly_deaths = []
#total_cases = []
#total_deaths = []
#month_case_total = 0
#month_death_total = 0
#state = Washington
#casecount = casecount



with open (file_path, 'r') as csvfile:
    csvreader = csv.reader(csvfile, delimiter =",")
    
    #skip header row
    next(csvreader, None)
    
    
    
    #loop through data splitting the date field 
    
    for row in csvreader:
        
                
        #split date and to date lists
        yr,m,d = row[0].split("-")
        year.append(yr)
        month.append(m)
        day.append(d)
        
        #add state to state list
        
    
        state.append(row[1])
        
                
        #add cases
        cases.append(row[3])
        
        #add deaths
        deaths.append(row[4])


#create a dictonary to se for the data frame
covid_dict = {"Year": year, "Month": month, "Day": day, "State": state, "Cases": cases, "Deaths": deaths}

#create data frame
reformated_date_df = pd.DataFrame(covid_dict)

reformated_date_df.head()


,Year,Month,Day,State,Cases,Deaths
0,2020,01,21,Washington,1,0
1,2020,01,22,Washington,1,0
2,2020,01,23,Washington,1,0
3,2020,01,24,Illinois,1,0
4,2020,01,24,Washington,1,0


In [22]:
reformated_date_df.dtypes

Year      object
Month     object
Day       object
State     object
Cases     object
Deaths    object
dtype: object

In [24]:
reformated_date_df = reformated_date_df.infer_objects()

In [ ]:
#

In [41]:
#calculate monthly totals for each state 

#group by state and month

State_group = reformated_date_df.groupby(['State', 'Month'], as_index = False)

State_group.last()


,State,Month,Year,Day,Cases,Deaths
0,Alabama,01,2021,31,459639,7688
1,Alabama,02,2021,28,493252,9929
2,Alabama,03,2021,31,515388,10554
3,Alabama,04,2021,29,527513,10887
4,Alabama,05,2020,31,17952,630
...,...,...,...,...,...,...
655,Wyoming,08,2020,31,3842,37
656,Wyoming,09,2020,30,5948,50
657,Wyoming,10,2020,31,13298,87
658,Wyoming,11,2020,30,33305,215


AttributeError: 'DataFrameGroupBy' object has no attribute 'reset_index'

,index,0,1
0,0,"(Alabama, 01)",Year Month Day State Cases Deaths ...
1,1,"(Alabama, 02)",Year Month Day State Cases Deaths ...
2,2,"(Alabama, 03)",Year Month Day State Cases Deaths ...
3,3,"(Alabama, 04)",Year Month Day State Cases Deaths ...
4,4,"(Alabama, 05)",Year Month Day State Cases Deaths 32...
...,...,...,...
655,655,"(Wyoming, 08)",Year Month Day State Cases Deaths 83...
656,656,"(Wyoming, 09)",Year Month Day State Cases Deaths 10...
657,657,"(Wyoming, 10)",Year Month Day State Cases Deaths 1...
658,658,"(Wyoming, 11)",Year Month Day State Cases Deaths 1...
